In [ ]:
#Imports y carga del dataset
import pandas as pd
import math
import io

pd.set_option("display.max_rows", None)

with open("movies_dataset/movies_metadata.csv", "r") as f:
    df_raw = pd.read_csv(f)

In [2]:
#Excluir columnas que no vamos a usar
exclude = ["homepage", "imdb_id", "original_title", "poster_path", "status", "video", "id"] 
df = df_raw.loc[:, df_raw.columns.difference(exclude)]

NameError: name 'df_raw' is not defined

In [1]:
#Comprobación de valores nulos
print(pd.isnull(df).any())
print()
print("Filas totales:", len(df))
print("Filas con algún valor nulo:", len([x for x in df.isnull().any(axis=1) if x]))

NameError: name 'pd' is not defined

In [6]:
#Procesado de la columna "adult"
#Aqui observamos que hay valores no booleanos que debemos eliminar, y además observamos que
#las películas con valor True son muy escasas en comparación con las que tienen valor False,
#por lo que vamos a eliminar las filas anómalas y quitar la columna "adult"
print(df["adult"].value_counts())
print()

drop_list = []
for i in range(len(df)):
    if df["adult"][i] != "False":
        drop_list.append(i)
df.drop(index=drop_list, inplace=True)
print(df["adult"].value_counts())
print()

df.drop(columns=["adult"], inplace=True)
print(list(df))

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
Name: adult, dtype: int64

False    45454
Name: adult, dtype: int64

['belongs_to_collection', 'budget', 'genres', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'tagline', 'title', 'vote_average', 'vote_count']


In [7]:
# title, tagline, overview y belongs_to_collection

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

replaces = [("i'm", "i am"),
            ("you're", "you are"),
            ("we're", "we are"),
            ("they're", "they are"),
            
            ("that's", "that is"),
            ("there's", "there is"),
            ("who's", "who is"), 
            ("what's", "what is"),
            
            ("isn't", "is not"),
            ("aren't", "are not"), 
            ("can't", "cannot"),
            ("haven't", "have not"),
            ("won't", "will not"),
            ("doesn't", "does not"),
            ("don't", "do not"),
            ("didn't", "did not"),
            ("couldn't", "could not"),
            ("shouldn't", "should not"),
            ("wasn't", "was not"),
            ("weren't", "were not"),
            ("hasn't", "has not"),
            ("hadn't", "had not"),
            
            ("we'd", "we would"),
            ("he'd", "he would"),
            ("you'd", "you would"),
            ("they'd", "they would"), 
            ("who'd", "who would"),
            ("wouldn't", "would not"),
            
            ("i'll", "i will"),
            ("you'll", "you will"),
            ("he'll", "he will"),
            ("she'll", "she will"),
            ("it'll", "it will"),
            ("we'll", "we will"),
            ("they'll", "they will"),
            
            ("i've", "i have"),
            ("you've", "you have"),
            ("we've", "we have"),
            ("they've", "they've"),
            ("&", " and ")
           ]

def sub_contractions(text):
    new_text = text
    for r in replaces:
        new_text = re.sub(f"{r[0]}", f"{r[1]}", new_text)
    return new_text

lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    if type(text) is float:  #Comprobando valores nulos, expresados como un float NaN
        ret_text = []
    else:
        new_text = text.lower()
        new_text = sub_contractions(new_text)
        new_text = re.sub("[^A-Za-zÀ-ÖØ-öø-ÿ0-9 ]", "", new_text)
        
        tok_text = word_tokenize(new_text)
        stop_words = list(stopwords.words("english"))
        tok_text = [word for word in tok_text if word not in stop_words]
        tok_text = [lemmatizer.lemmatize(word) for word in tok_text]
        ret_text = tok_text
        
        """ Processing feedback
        joint_text = " ".join(tok_text)
        print(new_text)
        print(joint_text)
        print() #"""
        
    return ret_text

In [9]:
df["preprocessed_tagline"] = df["tagline"].apply(preprocess_text)
df["preprocessed_tagline"]

0                                                       []
1                        [roll, dice, unleash, excitement]
2        [still, yelling, still, fighting, still, ready...
3                [friend, people, let, never, let, forget]
4                [world, back, normal, he, surprise, life]
5                              [los, angeles, crime, saga]
6           [cordially, invited, surprising, merger, year]
7                                     [original, bad, boy]
8                                   [terror, go, overtime]
9                                [limit, fear, substitute]
10                [powerful, man, world, one, thing, want]
11                                                      []
12                           [part, dog, part, wolf, hero]
13       [triumphant, victory, bitter, defeat, changed,...
14       [course, set, turning, back, prepare, weapon, ...
15                               [one, stay, top, forever]
16                               [lose, heart, come, sen

In [10]:
df["preprocessed_title"] = df["title"].apply(preprocess_text)
df["title"]

In [ ]:
df["preprocessed_overview"] = df["overview"].apply(preprocess_text)
df["preprocessed_overview"]

In [1]:
def preprocess_colection(coleccion):
    id_coleccion = -1
    if type(coleccion) != float:
        id_coleccion = re.search("'id' *: *([0-9]+)", coleccion)[1]
    return id_coleccion

df["preprocessed_colection"] = df["belongs_to_collection"].apply(preprocess_colection)
df["preprocessed_colection"]

NameError: name 'df' is not defined

In [12]:
#Fecha dejar a nulo, no eliminar
#Pais / idioma variables categoricas, agrupar en regiones y one hot encoding
#Objetivo es convertir en variables numericas, añadiendo columnas por one hot encoding